In [14]:
for pcap_file in pcap_files:
    file_path = os.path.join(pcap_dir, pcap_file)
    packets = rdpcap(file_path)
    rows = []
    for pkt in packets:
        # Get the highest layer protocol name
        proto = pkt.lastlayer().name if hasattr(pkt, "lastlayer") else pkt.name

        # Default to None
        src_ip = dst_ip = None

        # Try to extract IP addresses if present
        if pkt.haslayer('IP'):
            src_ip = pkt['IP'].src
            dst_ip = pkt['IP'].dst
        elif pkt.haslayer('IPv6'):
            src_ip = pkt['IPv6'].src
            dst_ip = pkt['IPv6'].dst
        else:
            # fallback to MAC addresses if no IP layer
            src_ip = pkt[0].src if hasattr(pkt[0], 'src') else None
            dst_ip = pkt[0].dst if hasattr(pkt[0], 'dst') else None

        row = {
            'time': pkt.time,
            'src': src_ip,
            'dst': dst_ip,
            'proto': proto,
            'len': len(pkt)
        }
        rows.append(row)
    df = pd.DataFrame(rows)
    pcap_dfs[pcap_file] = df


In [16]:
# Get unique protocol, source, and destination combinations and their counts for every file
protocol_summaries = {}
for fname, df in pcap_dfs.items():
    summary = df.groupby(['proto', 'src', 'dst']).size().reset_index(name='count')
    protocol_summaries[fname] = summary
    print(f"Summary for {fname}:")
    print(summary)
    
    
    


Columns in adversary6.pcap: ['time', 'src', 'dst', 'proto', 'len']
Summary for adversary6.pcap:
                               proto                       src  \
0                                ARP         00:50:56:ae:b3:f0   
1                                ARP         00:50:56:ae:b3:f0   
2                                DNS              192.168.3.60   
3                                DNS              192.168.3.60   
4                                DNS                   8.8.8.8   
5                                DNS  fe80::250:56ff:feae:b3f0   
6                           Kerberos              192.168.3.60   
7                   LDAP_SASL_Buffer              192.168.3.27   
8                   LDAP_SASL_Buffer              192.168.3.60   
9          NBNS Node Status Response              192.168.3.23   
10                NBNS query request              192.168.3.60   
11               NBNS query response              192.168.3.23   
12               NBNS query response          

In [19]:
for df in pcap_dfs:
    print(df)
    
    df_summary = pcap_dfs[df].groupby(['proto', 'src', 'dst']).size().reset_index(name='count')
    print(df_summary[['proto', 'src', 'dst']])

adversary6.pcap
                               proto                       src  \
0                                ARP         00:50:56:ae:b3:f0   
1                                ARP         00:50:56:ae:b3:f0   
2                                DNS              192.168.3.60   
3                                DNS              192.168.3.60   
4                                DNS                   8.8.8.8   
5                                DNS  fe80::250:56ff:feae:b3f0   
6                           Kerberos              192.168.3.60   
7                   LDAP_SASL_Buffer              192.168.3.27   
8                   LDAP_SASL_Buffer              192.168.3.60   
9          NBNS Node Status Response              192.168.3.23   
10                NBNS query request              192.168.3.60   
11               NBNS query response              192.168.3.23   
12               NBNS query response              192.168.3.60   
13               NBNS status request              192.168.3.